In [ ]:
import numpy as np
import pandas as pd
import keras as k
import tensorflow as tf
import os
import zipfile
import cv2
import matplotlib as mpl
import matplotlib.pyplot as plt
from keras.layers import Conv2D
from keras.layers import LeakyReLU,Activation,Input,Dropout,MaxPool2D,Flatten,Dense,BatchNormalization
from keras.optimizers import Adam

In [ ]:
zip_files=['train','test1']
for i in zip_files:
    with zipfile.ZipFile("../input/dogs-vs-cats/"+i+".zip",'r') as z:
        z.extractall(".")
        z.printdir()
        print(f"{i} unzipped")

In [ ]:
main_dir = "/kaggle/working/"
train_dir = "train"
path = os.path.join(main_dir,train_dir)

In [ ]:
for p in os.listdir(path):
    category=p.split(".")[0]
    image_arr=cv2.imread(os.path.join(path,p),cv2.IMREAD_GRAYSCALE)
    img_arr=cv2.resize(image_arr,dsize=(84,84))
    plt.imshow(img_arr,cmap="gray")
    break
#     print(category)

In [ ]:
x=[]
y=[]
convert=lambda category: int(category=="dog")
for p in os.listdir(path):
    category=p.split(".")[0]
    category=convert(category)
    img_array=cv2.imread(os.path.join(path,p),cv2.IMREAD_GRAYSCALE)
    img_arr=cv2.resize(img_array,dsize=(84,84))
    x.append(img_arr)
    y.append(category)
x=np.array(x)
y=np.array(y)

In [ ]:
x=x/255

In [ ]:
x=np.array(x).reshape(-1,84,84,1)
y=np.array(y).reshape(-1,1)
print(x.shape,y.shape)

In [ ]:
# from sklearn.model_selection import train_test_split
# x_train,y_train,x_val,y_val=train_test_split(x,y,test_size=0.2,random_state=42)
# print(x_train.shape,y_train.shape,x_val.shape,y_val.shape)

In [ ]:
model = k.Sequential()

model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(84,84,1)))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid')) # 2 because we have cat and dog classes


In [ ]:
initial_lr=0.01
loss="binary_crossentropy"
model.compile(Adam(lr=initial_lr),loss=loss,metrics=['accuracy'])
model.summary()

In [ ]:
epochs=30
batch_size=64
# print(x_val.shape)
history_1=model.fit(x,y,batch_size=batch_size,epochs=epochs,validation_split=0.2)

In [ ]:
plt.plot(history_1.history['loss'],label="training set loss")
plt.plot(history_1.history['val_loss'],label="val set size")
plt.legend()
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')

In [ ]:
main_dir = "/kaggle/working/"
test_dir = "test1"
path = os.path.join(main_dir,test_dir)

In [ ]:
x_test=[]
id_line=[]
for p in os.listdir(path):
    id_line.append(p.split(".")[0])
    img_array=cv2.imread(os.path.join(path,p),cv2.IMREAD_GRAYSCALE)
    img_arr=cv2.resize(img_array,dsize=(84,84))
    x_test.append(img_arr)
x_test=np.array(x_test)

In [ ]:
x_test=x_test/255
x_test=np.array(x_test).reshape(-1,84,84,1)
predictions=model.predict(x_test)

In [ ]:
predicted_val=[int(round(p[0])) for p in predictions]

In [ ]:
submission_df = pd.DataFrame({'id':id_line, 'label':predicted_val})

In [ ]:
submission_df.to_csv("submission.csv",index=False)